# BMSSP Solver Runtime Experiments

This notebook runs performance experiments on the BMSSP (Batch Multi-Source Shortest Path) solver to analyze runtime scaling with respect to graph size and edge density.

## Configuration

Edit the parameters below to customize your experiments.

In [ ]:
# ===== EXPERIMENT PARAMETERS (EDIT THESE) =====

SOLVER_PATH = "../bmssp_solver"
NUM_TRIALS = 5  # Number of trials per configuration

# Node scaling experiment
NODE_COUNTS = [10000, 20000, 40000, 80000, 160000]
EDGE_MULTIPLIER = 4  # m = EDGE_MULTIPLIER * n

# Edge density experiment
FIXED_NODES = 20000
EDGE_MULTIPLIERS = [2, 4, 8, 16, 32, 64]

# Graph generation
WEIGHT_MIN = 0.1
WEIGHT_MAX = 100.0

## Imports and Setup

In [ ]:
import subprocess
import random
import re
import os
import statistics
import tempfile
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

# Enable inline plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

# Verify solver exists
solver_abs_path = os.path.abspath(SOLVER_PATH)
if os.path.isfile(solver_abs_path):
    print(f"Solver found: {solver_abs_path}")
else:
    print(f"WARNING: Solver not found at {solver_abs_path}")
    print("Please build the project first: cd .. && cmake . && make")

## Graph Generator

Generates random connected directed graphs. Ensures connectivity by first creating a spanning tree, then adding random edges.

In [ ]:
def generate_connected_graph(n, m, seed=None, weight_min=WEIGHT_MIN, weight_max=WEIGHT_MAX):
    """
    Generate a random connected directed graph.
    
    Args:
        n: Number of nodes
        m: Number of edges (must be >= n-1)
        seed: Random seed for reproducibility
        weight_min: Minimum edge weight
        weight_max: Maximum edge weight
    
    Returns:
        List of tuples (u, v, w)
    """
    if seed is not None:
        random.seed(seed)
    
    if m < n - 1:
        raise ValueError(f"Need at least {n-1} edges for {n} nodes to be connected")
    
    edges = []
    edge_set = set()
    
    # Step 1: Create spanning tree for connectivity
    nodes = list(range(n))
    random.shuffle(nodes)
    
    for i in range(1, n):
        parent = random.randint(0, i - 1)
        u, v = nodes[parent], nodes[i]
        w = random.uniform(weight_min, weight_max)
        edges.append((u, v, w))
        edge_set.add((u, v))
    
    # Step 2: Add remaining random edges
    remaining = m - (n - 1)
    attempts = 0
    max_attempts = remaining * 100
    
    while len(edges) < m and attempts < max_attempts:
        u = random.randint(0, n - 1)
        v = random.randint(0, n - 1)
        
        if u != v and (u, v) not in edge_set:
            w = random.uniform(weight_min, weight_max)
            edges.append((u, v, w))
            edge_set.add((u, v))
        
        attempts += 1
    
    random.shuffle(edges)
    return edges


def write_graph_to_file(filepath, n, edges, source=0):
    """Write graph in BMSSP solver input format."""
    with open(filepath, 'w') as f:
        f.write(f"{n} {len(edges)}\n")
        for u, v, w in edges:
            f.write(f"{u} {v} {w:.4f}\n")
        f.write(f"{source}\n")


# Test the generator
test_edges = generate_connected_graph(100, 400, seed=42)
print(f"Generated test graph: 100 nodes, {len(test_edges)} edges")
print(f"Sample edges: {test_edges[:3]}")

## Experiment Runner

Functions to run the solver and collect timing data.

In [ ]:
def extract_timing(output):
    """Extract timing in milliseconds from solver output."""
    match = re.search(r'BMSSP Time:\s*([\d.]+)\s*ms', output)
    if match:
        return float(match.group(1))
    raise ValueError(f"Could not extract timing from output: {output[:200]}")


def run_solver(input_file, timeout=300):
    """
    Run the BMSSP solver on an input file.
    
    Returns:
        Tuple of (timing_ms, success)
    """
    try:
        with open(input_file, 'r') as f:
            result = subprocess.run(
                [solver_abs_path],
                stdin=f,
                capture_output=True,
                text=True,
                timeout=timeout
            )
        
        if result.returncode != 0:
            print(f"  Warning: Non-zero exit code: {result.returncode}")
            return 0.0, False
        
        timing = extract_timing(result.stdout)
        return timing, True
        
    except subprocess.TimeoutExpired:
        print(f"  Warning: Solver timed out")
        return 0.0, False
    except Exception as e:
        print(f"  Error: {e}")
        return 0.0, False


def run_experiment(n, m, num_trials, base_seed=0):
    """
    Run multiple trials of the solver with given graph parameters.
    
    Returns:
        dict with statistics and all timings
    """
    timings = []
    
    with tempfile.TemporaryDirectory() as tmpdir:
        for trial in range(num_trials):
            seed = base_seed + trial
            input_file = os.path.join(tmpdir, f"graph_{trial}.in")
            
            # Generate graph
            edges = generate_connected_graph(n, m, seed=seed)
            write_graph_to_file(input_file, n, edges)
            
            # Run solver
            timing, success = run_solver(input_file)
            
            if success:
                timings.append(timing)
                print(f"    Trial {trial + 1}/{num_trials}: {timing:.2f} ms")
            else:
                print(f"    Trial {trial + 1}/{num_trials}: FAILED")
    
    if not timings:
        return None
    
    return {
        'timings': timings,
        'mean': statistics.mean(timings),
        'std': statistics.stdev(timings) if len(timings) > 1 else 0,
        'min': min(timings),
        'max': max(timings),
        'median': statistics.median(timings)
    }

## Node Scaling Experiment

Measure how runtime scales with the number of nodes (fixed edge density).

In [ ]:
print("="*60)
print("NODE SCALING EXPERIMENT")
print(f"Edge density: m = {EDGE_MULTIPLIER} * n")
print(f"Trials per configuration: {NUM_TRIALS}")
print("="*60)

node_scaling_results = []

for n in NODE_COUNTS:
    m = EDGE_MULTIPLIER * n
    print(f"\nConfiguration: n={n:,}, m={m:,}")
    
    result = run_experiment(n, m, NUM_TRIALS, base_seed=n)
    
    if result:
        node_scaling_results.append({
            'nodes': n,
            'edges': m,
            **result
        })
        print(f"  Summary: mean={result['mean']:.2f}ms, std={result['std']:.2f}ms")

print("\n" + "="*60)
print("Node scaling experiment complete!")
print("="*60)

## Node Scaling Plot

In [ ]:
if node_scaling_results:
    nodes = [r['nodes'] for r in node_scaling_results]
    mean_times = [r['mean'] for r in node_scaling_results]
    std_times = [r['std'] for r in node_scaling_results]
    
    fig, ax = plt.subplots(figsize=(10, 7))
    
    # Plot with error bars
    ax.errorbar(nodes, mean_times, yerr=std_times,
                fmt='o-', capsize=5, capthick=2,
                markersize=8, linewidth=2,
                color='#2E86AB', ecolor='#A23B72',
                label='BMSSP Runtime')
    
    # Add individual data points
    for r in node_scaling_results:
        xs = [r['nodes']] * len(r['timings'])
        ax.scatter(xs, r['timings'], alpha=0.3, s=20, color='#2E86AB')
    
    ax.set_xlabel('Number of Nodes (n)', fontsize=14)
    ax.set_ylabel('Runtime (ms)', fontsize=14)
    ax.set_title(f'BMSSP Runtime vs. Graph Size\n(Edge Density: m = {EDGE_MULTIPLIER}n)', fontsize=16)
    
    ax.set_xscale('log')
    ax.set_yscale('log')
    
    ax.set_xticks(nodes)
    ax.set_xticklabels([f'{n//1000}K' for n in nodes])
    
    ax.legend(fontsize=12)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No results to plot. Run the node scaling experiment first.")

## Edge Density Experiment

Measure how runtime scales with edge density (fixed number of nodes).

In [ ]:
print("="*60)
print("EDGE DENSITY EXPERIMENT")
print(f"Fixed nodes: n = {FIXED_NODES:,}")
print(f"Edge multipliers: {EDGE_MULTIPLIERS}")
print(f"Trials per configuration: {NUM_TRIALS}")
print("="*60)

edge_density_results = []

for multiplier in EDGE_MULTIPLIERS:
    n = FIXED_NODES
    m = multiplier * n
    print(f"\nConfiguration: n={n:,}, m={m:,} (multiplier={multiplier})")
    
    result = run_experiment(n, m, NUM_TRIALS, base_seed=multiplier * 1000)
    
    if result:
        edge_density_results.append({
            'nodes': n,
            'edges': m,
            'multiplier': multiplier,
            **result
        })
        print(f"  Summary: mean={result['mean']:.2f}ms, std={result['std']:.2f}ms")

print("\n" + "="*60)
print("Edge density experiment complete!")
print("="*60)

## Edge Density Plot

In [ ]:
if edge_density_results:
    multipliers = [r['multiplier'] for r in edge_density_results]
    mean_times = [r['mean'] for r in edge_density_results]
    std_times = [r['std'] for r in edge_density_results]
    
    fig, ax = plt.subplots(figsize=(10, 7))
    
    ax.errorbar(multipliers, mean_times, yerr=std_times,
                fmt='s-', capsize=5, capthick=2,
                markersize=8, linewidth=2,
                color='#E94F37', ecolor='#393E41',
                label='BMSSP Runtime')
    
    # Add individual data points
    for r in edge_density_results:
        xs = [r['multiplier']] * len(r['timings'])
        ax.scatter(xs, r['timings'], alpha=0.3, s=20, color='#E94F37')
    
    ax.set_xlabel('Edge Multiplier (m / n)', fontsize=14)
    ax.set_ylabel('Runtime (ms)', fontsize=14)
    ax.set_title(f'BMSSP Runtime vs. Edge Density\n(Fixed n = {FIXED_NODES:,})', fontsize=16)
    
    ax.set_xscale('log', base=2)
    ax.set_yscale('log')
    
    ax.set_xticks(multipliers)
    ax.set_xticklabels([str(m) for m in multipliers])
    
    ax.legend(fontsize=12)
    ax.grid(True, alpha=0.3)
    
    # Secondary x-axis showing actual edge counts
    ax2 = ax.twiny()
    ax2.set_xscale('log', base=2)
    ax2.set_xlim(ax.get_xlim())
    ax2.set_xticks(multipliers)
    ax2.set_xticklabels([f'{m*FIXED_NODES//1000}K' for m in multipliers])
    ax2.set_xlabel('Number of Edges (m)', fontsize=12)
    
    plt.tight_layout()
    plt.show()
else:
    print("No results to plot. Run the edge density experiment first.")

## Combined Analysis

In [ ]:
if node_scaling_results and edge_density_results:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Left plot: Node scaling
    nodes = [r['nodes'] for r in node_scaling_results]
    mean_times = [r['mean'] for r in node_scaling_results]
    std_times = [r['std'] for r in node_scaling_results]
    
    ax1.errorbar(nodes, mean_times, yerr=std_times,
                 fmt='o-', capsize=5, capthick=2,
                 markersize=8, linewidth=2,
                 color='#2E86AB', ecolor='#A23B72')
    
    ax1.set_xlabel('Number of Nodes (n)', fontsize=12)
    ax1.set_ylabel('Runtime (ms)', fontsize=12)
    ax1.set_title(f'Runtime vs. Graph Size (m = {EDGE_MULTIPLIER}n)', fontsize=14)
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.set_xticks(nodes)
    ax1.set_xticklabels([f'{n//1000}K' for n in nodes])
    ax1.grid(True, alpha=0.3)
    
    # Right plot: Edge density
    multipliers = [r['multiplier'] for r in edge_density_results]
    mean_times2 = [r['mean'] for r in edge_density_results]
    std_times2 = [r['std'] for r in edge_density_results]
    
    ax2.errorbar(multipliers, mean_times2, yerr=std_times2,
                 fmt='s-', capsize=5, capthick=2,
                 markersize=8, linewidth=2,
                 color='#E94F37', ecolor='#393E41')
    
    ax2.set_xlabel('Edge Multiplier (m / n)', fontsize=12)
    ax2.set_ylabel('Runtime (ms)', fontsize=12)
    ax2.set_title(f'Runtime vs. Edge Density (n = {FIXED_NODES:,})', fontsize=14)
    ax2.set_xscale('log', base=2)
    ax2.set_yscale('log')
    ax2.set_xticks(multipliers)
    ax2.set_xticklabels([str(m) for m in multipliers])
    ax2.grid(True, alpha=0.3)
    
    plt.suptitle('BMSSP Solver Performance Analysis', fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print("Run both experiments first to see the combined analysis.")

## Results Summary

In [ ]:
print("="*70)
print("RESULTS SUMMARY")
print("="*70)

if node_scaling_results:
    print("\nNode Scaling Results:")
    print(f"{'Nodes':>10} {'Edges':>12} {'Mean (ms)':>12} {'Std (ms)':>10} {'Min':>10} {'Max':>10}")
    print("-" * 70)
    for r in node_scaling_results:
        print(f"{r['nodes']:>10,} {r['edges']:>12,} {r['mean']:>12.2f} {r['std']:>10.2f} {r['min']:>10.2f} {r['max']:>10.2f}")

if edge_density_results:
    print(f"\nEdge Density Results (n = {FIXED_NODES:,}):")
    print(f"{'Multiplier':>10} {'Edges':>12} {'Mean (ms)':>12} {'Std (ms)':>10} {'Min':>10} {'Max':>10}")
    print("-" * 70)
    for r in edge_density_results:
        print(f"{r['multiplier']:>10} {r['edges']:>12,} {r['mean']:>12.2f} {r['std']:>10.2f} {r['min']:>10.2f} {r['max']:>10.2f}")

## Save Results (Optional)

Run this cell to save results to files.

In [ ]:
import json
import csv

# Create output directories
Path('results').mkdir(exist_ok=True)
Path('plots').mkdir(exist_ok=True)

# Save JSON results
if node_scaling_results:
    with open('results/node_scaling.json', 'w') as f:
        json.dump(node_scaling_results, f, indent=2)
    print("Saved: results/node_scaling.json")

if edge_density_results:
    with open('results/edge_density.json', 'w') as f:
        json.dump(edge_density_results, f, indent=2)
    print("Saved: results/edge_density.json")

# Save CSV summaries
if node_scaling_results:
    with open('results/node_scaling.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['nodes', 'edges', 'mean_ms', 'std_ms', 'min_ms', 'max_ms', 'median_ms'])
        for r in node_scaling_results:
            writer.writerow([r['nodes'], r['edges'], r['mean'], r['std'], r['min'], r['max'], r['median']])
    print("Saved: results/node_scaling.csv")

if edge_density_results:
    with open('results/edge_density.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['nodes', 'edges', 'multiplier', 'mean_ms', 'std_ms', 'min_ms', 'max_ms', 'median_ms'])
        for r in edge_density_results:
            writer.writerow([r['nodes'], r['edges'], r['multiplier'], r['mean'], r['std'], r['min'], r['max'], r['median']])
    print("Saved: results/edge_density.csv")

print("\nDone! Check the 'results' directory for saved files.")